In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bocd_features = pd.read_hdf("D:/oxford/Dissertation/cs_strats/features/bocd_feat_backups/bocd_per_stock_mle_asof_top1k_more_feats_v2_hazard_100_days.h5", 
                            key='bocd_per_stock_mle_asof_top1k_more_feats_v2_hazard_100_days')

bocd_features = pd.read_hdf("features/bocd_per_stock_feats.h5", 
                            key='bocd_per_stock_feats')
bocd_features['permno'] = bocd_features['permno'].astype(int)

In [ ]:
ltr_df = pd.read_hdf("features/ltr_df.h5", 'ltr_df')

In [4]:
ltr_df_w_bocd_feat = pd.merge(
    ltr_df,
    bocd_features,
    how='left',
    on=['date', 'permno']
)

ltr_df_w_bocd_feat.dropna(inplace=True)
ltr_df_w_bocd_feat.columns = ltr_df_w_bocd_feat.columns.str.replace(
    r'^P_r_le_', 'Pr_le_', regex=True
)

In [ ]:
# Save memoery
del ltr_df

In [9]:
def cross_section_winsor_z(df, feature_cols, date_col='date',
                           lower_q=0.01, upper_q=0.99, eps=1e-8):
    df = df.copy()
    for f in feature_cols:
        # Apply per date
        grouped = df.groupby(date_col)[f]
        # Compute quantiles
        q_low  = grouped.transform(lambda x: x.quantile(lower_q))
        q_high = grouped.transform(lambda x: x.quantile(upper_q))
        clipped = df[f].clip(q_low, q_high)
        mean = clipped.groupby(df[date_col]).transform('mean')
        std  = clipped.groupby(df[date_col]).transform('std').replace(0, np.nan)
        df[f + '_z'] = (clipped - mean) / (std + eps)
    return df

In [10]:
ltr_df_w_bocd_feat = cross_section_winsor_z(ltr_df_w_bocd_feat, 
                                            ['p_t', 'E_rt', 'dE_rt', 'Pr_le_5', 'Pr_le_10',
                                             'Pr_le_14', 'Pr_le_21', 'Pr_le_42', 'Pr_le_63', 'Pr_le_126',
                                             'Pr_le_252', 'log_pred', 'H_rt', 'Var_rt', 'r_med'])

In [ ]:
# CAUTION: THIS OVERWRITES THE EXISTING ltr_hdf, hence it is commented out (uncomment out after you verify it is safe)
ltr_df_w_bocd_feat.to_hdf(
    'features/ltr_df.h5',
    key='ltr_df',
    mode='w',
    complib='blosc',
    complevel=9
)